## Import siibra

In [ ]:
# first off, update siibra to latest release
!pip install -U siibra

In [ ]:
import siibra
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0a08')
import os
import matplotlib
%matplotlib inline

In [ ]:
# We populate the cache with common data items here, so we need not wait later on.
# This is not usually needed - siibra fetches data only as needed.
with siibra.QUIET:
    siibra.warm_cache()

## Accessing brain parcellations

Preconfigured reference parcellations are stored in the instance table `siibra.parcellations`. 
The configuration is retrieved automatically from an github repository that we maintain with siibra.
Instance table provide a tabular overview of their elements with the `dataframe` function, which returns a [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) - a rich object with functions similar to Excel.

In [ ]:
siibra.parcellations.dataframe

Elements in an instance table can be accessed in a couple of ways, in particular

 - by iterating over all instances
 - by fuzzy matching of keyword or name with the index operator `[.]` or the `get()` method
 - by tab-completion

Let's use keyword matching to retrieve the most recent Julich Brain parcellation.

In [ ]:
julichbrain = siibra.parcellations.get('julich')
julichbrain

There is also an instance table of atlases, which we could use to access the parcellations linked with the human atlas.

In [ ]:
siibra.atlases.get('human').parcellations

In [ ]:
# let's look at some metadata
print(f"Name:     {julichbrain.name}")
print(f"Id:       {julichbrain.id}")
print(f"Modality: {julichbrain.modality}\n")
print(f"{julichbrain.description}\n")
for p in julichbrain.publications:
    print(p['citation'])

The resulting parcellation is a semantic object. It represents the region hierarchy of the parcellation.
We can find regions by name using the `find` function. If we know unique keywords and expect a single match, we can also use `get`.

In [ ]:
for region in julichbrain.find('v1'):
    print(region.name)

As you see, areas often appear three times: Julich-Brain defines them separately for the left and right hemisphere, and additionally defines a common parent region. In fact the parent object represents the corresponding subtree. We can more easily access individual regions by using `get_region` instead of `find_regions`. This method assumes the region specification is unique, and either returns a single region object or fails. If it finds multiple matches, it will try if they have a common parent.

In [ ]:
# the whole amygdala subtree
julichbrain.get_region('v1')

You may output the subtree anchored at a given region, if any, using `Region.tree2str()`. This is useful to inspect a region object.

In [ ]:
occ = julichbrain.get_region('occipital cortex')
print(occ.tree2str())

## Accessing parcellation maps

A parcellation map or region map is a spatial object corresponding to a parcellation. 
We can access maps with the `get_map` function of parcellation objects.
Since parcellations may provide maps in different spaces, `siibra` expects you to specify the space. 
Note: Preconfigured reference spaces are managed in another instance table - `siibra.spaces` (you might have guessed it). 


Let's access the maximum probability map of Julich-Brain in the MNI152 space to see how that works.

In [ ]:
julich_mpm = julichbrain.get_map(space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
mpm

## Fetching the actual image of a parcellation map
The returned map provides all information required to fetch the actual image.
To access it we need to retrieve the actual data using the `fetch()` method, which returns a Nifti1Image object.
This step is separate for two reasons:
- The parcellation map is more than just the image - it provides information about the space and parcellation of the map, and possibly multiple resource where the data is stored.
- `siibra` uses a lazy strategy for data loading. `fetch` is the typical last step to actually retrieve the underlying content.

We can use the wonderful `nilearn` library for plotting the map. It plots in the MNI152 space by default, so as long as we work in this space plotting is simple enough.

Some parcellations (and other 3D volumes) are split into multiple fragments represented in separate image volumes. For Julich-Brain 2.9, each hemisphere is in a different fragment. We can fetch individual fragments, but if no fragment is specified, siibra will merge the available ones into a single volume:

In [ ]:
from nilearn import plotting
cmap = julich_mpm.get_colormap()
plotting.plot_roi(julich_mpm.fetch(), cmap=cmap, title=julich_mpm.parcellation.name)

## Fetching probability maps

Julich-Brain, like some other parcellations, is a probabilistic parcellation. The labelled volumes in the maximum probability map (mpm) above are only a summary representation, displaying for each voxel the brain region of highest probability. 
Each region is additionally available as a probability map, which provides statistical information in the reference space for each particular region.

We received the labelled volumes above because `siibra` uses labelled volumes as the default map type. 
To retrieve probability maps, we explicitly request `siibra.MapType.STATISTICAL` as maptype from the parcellation.
It returns a sparse map representation, since the set of all probability maps contains several 100 of NIfTI volumes with mostly empty voxels.

In [ ]:
julich_pmaps = julichbrain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.STATISTICAL
)
julich_pmaps

To access the probability maps, we will call fetch again. However, this time, we need to specify a region.
The sparse representation will then generate a (dense) Nifti1Image which we can use as expected.
Plotting of probability maps works nicely with nilearn's `plot_stat_map`.

In [ ]:
pmap = julich_pmaps.fetch(region='hoc5 right')
plotting.plot_stat_map(pmap, title=f'hOc5 right of {julich_pmaps.parcellation.name}')

In the background, `siibra` uses an index to identify regions in a parcellation map.
The index informs about the image volume and the label used to map the region.
Usually we don't need to, but we can request and use these indices as well for fetching.
We will see that a region in the probability map is indexed by the volume, not by a label.

In [ ]:
index = julich_pmaps.get_index(region='hoc5 right')
index

This is different if we request the index of the same region in the maximum probability map, which is a labelled parcellation and represents all regions by their voxel label in the same volume:

In [ ]:
index = julich_mpm.get_index(region='hoc5 right')
index

As mentioned before, while not recommended, we can also use this index to fetch from the map instead of using a region or region name:

In [ ]:
pmap = julich_pmaps.fetch(index=index)
plotting.plot_stat_map(pmap, title=f'hOc5 right of {julich_pmaps.parcellation.name}')

If we request a specific region when fetching from the labelled map, `siibra` will construct a binary mask of the region. This is different in shape from the probabilistic maps, but of course sits at the same location.

In [ ]:
mask = julich_mpm.fetch(region='hoc5 right')
plotting.plot_roi(mask, title=f'hOc5 right of {julich_pmaps.parcellation.name}')

## Extracting volumes of interest from high resolution

Accessing image volumes is at the heart of `siibra`, and also works for high resolution images such as the BigBrain model. 

BigBrain is a reference space in `siibra`, and the corresponding image is the template of that space.
Getting a template from a space corresonds to getting the map of a parcellation - we call the `get_template` method of the space object.

To get access to the image data of the template, we use `fetch` again on the template object.
However, fetching BigBrain at full resolution is not a good idea - it is a 1TByte dataset. 
`siibra` will therefore by default fetch a downscaled version!

In [ ]:
bigbrain = siibra.spaces.get('bigbrain').get_template()
bigbrain_img = bigbrain.fetch()
plotting.plot_img(bigbrain_img, cmap='gray')

If we request the full resolution, `siibra` will complain and choose a larger but feasible resolution.

In [ ]:
bigbrain.fetch(resolution_mm=0.02)

To work with full resolution data, we typically fetch volumes of interest only.
`siibra` represents these as bounding boxes (`siibra.locations.BoundingBox`).
Bounding boxes are one type of locations provided by `siibra`, and all locations are uniquely associated to a reference space.
We construct a bounding box in BigBrain space by using the min and max point (-3.979, -61.256, 3.906) and (5.863, -55.356, -2.487):

In [ ]:
voi = siibra.locations.BoundingBox(
    (-3.979, -61.256, 3.906),
    (5.863, -55.356, -2.487),
    space='bigbrain'
)

This bounding box can be used to fetch a full resolution chunk from BigBrain.
To look around in the chunk, nilearn's `view_img` is nice!

In [ ]:
bigbrainchunk = bigbrain.fetch(resolution_mm=-1, voi=voi)
plotting.view_img(bigbrainchunk, bg_img=None, cmap='gray')

The resulting image chunk sits properly in its reference space, so we can also plot it on top of the low-resolution whole brain image that we fetched already above.

In [ ]:
plotting.plot_roi(bigbrainchunk, bg_img=bigbrain_img)

We can apply the same bounding box to extract chunks from other objects in the same space, like parcellation maps. 
Here we use the cortical layer maps in BigBrain space (in labelled map format), and download them in full resolution.
For the superimposition, we can use `view_img` with reduced `opacity`.

In [ ]:
layermap = siibra.parcellations.get('cortical layers').get_map(space='bigbrain')
mask = layermap.fetch(resolution_mm=0.16, voi=voi, fragment='left')
plotting.view_img(mask, bg_img=bigbrainchunk, opacity=.1, symmetric_cmap=False)